In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
df= pd.read_csv('SMSSpamCollection.tsv',sep='\t')

In [4]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [5]:
df.describe()

,length,punct
count,5572.000000,5572.000000
mean,80.489950,4.177495
std,59.942907,4.623919
min,2.000000,0.000000
25%,36.000000,2.000000
50%,62.000000,3.000000
75%,122.000000,6.000000
max,910.000000,133.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
 2   length   5572 non-null   int64 
 3   punct    5572 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 174.3+ KB


Checking the Null Values

In [7]:
df['label'].value_counts() 

label
ham     4825
spam     747
Name: count, dtype: int64

Spliting Data into Test & Train

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = df['message']
y = df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

CountVectorizer

It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text

In [12]:
count_vect=CountVectorizer()

In [13]:
X[:2]

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
Name: message, dtype: object

Fit the data to vectorizer

In [14]:
# Fit the vectorizer to the data  (build a vocab, count the number of words)
count_vect.fit(X_train)
X_train_counts=count_vect.transform(X_train)

In [15]:
X_train_counts

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 52150 stored elements and shape (3900, 7263)>

In [16]:
X_train.shape

(3900,)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

In [18]:
tfidf_transformer=TfidfTransformer()

In [19]:

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [20]:
X_train_tfidf.shape

(3900, 7263)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

The TfidfVectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allow you to encode new documents.

In [22]:
vectorizer=TfidfVectorizer()

In [23]:
X_train_tfidf=vectorizer.fit_transform(X_train)

In [24]:
from sklearn.svm import LinearSVC

The objective of a Linear SVC (Support Vector Classifier) is to fit to the data you provide, returning a "best fit" hyperplane that divides, or categorizes, your data

In [25]:
clf=LinearSVC()

In [26]:
clf.fit(X_train_tfidf,y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None


In [27]:
from sklearn.pipeline import Pipeline

In [28]:
text_clf=Pipeline([('tfidf', TfidfVectorizer()), ('clf',LinearSVC())])

In [29]:
text_clf.fit(X_train, y_train)

,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [30]:
predictions=text_clf.predict(X_test)

In [31]:
predictions=text_clf.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

In [33]:
print(confusion_matrix(y_test, predictions))

[[1445    3]
 [  10  214]]


In [34]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1448
        spam       0.99      0.96      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.98      0.98      1672
weighted avg       0.99      0.99      0.99      1672



In [35]:
from sklearn import metrics

In [36]:
metrics.accuracy_score(y_test,predictions)

0.9922248803827751

In [37]:
text_clf.predict(["Hello, my Name is Abhishek?"])

array(['ham'], dtype=object)

In [38]:
text_clf.predict(["Free car wash with mobile phone"])

array(['spam'], dtype=object)

In [39]:
text_clf.predict(["Hello, here is Pavan! Grab your free mobile phone now"])

array(['spam'], dtype=object)